<a href="https://colab.research.google.com/github/jakobbaumgartner/CNN-HotDog-Classifier/blob/master/16_main_with_data_augmentation_loops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from keras import backend as K

%reload_ext tensorboard


import time


In [2]:
tf.keras.__version__

'2.4.0'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm


Če imam izbrano GPU:

In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
# categories = ["hot_dog", "not_hot_dog"]
train_datadir = "drive/MyDrive/Colab Notebooks/seefood2/train"
validation_datadir = "drive/MyDrive/Colab Notebooks/seefood2/validation"
# os.getcwd()


Zaradi majhnega nabora podatkov, s katerimi obratujem, bom s pomočjo ImageDataGenerator podatke razširil tako, da bom slike rotiral, zrcalil, raztegoval.

 *The basic CNN structure is as follows: Convolution -> Pooling -> Convolution -> Pooling -> Fully Connected Layer -> Output*

*Convolution is the act of taking the original data, and creating feature maps from it.Pooling is down-sampling, most often in the form of "max-pooling," where we select a region, and then take the maximum value in that region, and that becomes the new value for the entire region. Fully Connected Layers are typical neural networks, where all nodes are "fully connected." The convolutional layers are not fully connected like a traditional neural network.* 


Na tej točki začnemo zares načrtovati nevronsko mrežo.

In [ ]:


batch_sizes = [16, 32]

imsizees = [80, 100, 150]

number_of_conv_layers = [1,2,3]
# number_of_dense_layers = [1,2,3]
size_of_conv_layers = [16,32,64, 128]
size_of_dense_layers = [16,32,64]



for batch_size in batch_sizes:
    for imsizee in tqdm(imsizees):
        for number_of_conv_layer in number_of_conv_layers:
            for size_of_conv_layer in size_of_conv_layers:
                for size_of_dense_layer in size_of_dense_layers:

                    print("\n------------------------------------------")
                    print("\nbatch_size: " + str(batch_size) + "  imsizee: " + str(imsizee) + "  number_of_conv_layer: " + str(number_of_conv_layer) + "  size_of_conv_layer: " + str(size_of_conv_layer) + "  size_of_dense_layer: " + str(size_of_dense_layer))

                    train_datagen = ImageDataGenerator(
                            rotation_range=40,  # rotiramo
                            width_shift_range=0.2, # raztezamo
                            height_shift_range=0.2,
                            rescale=1./255, # normaliziramo
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=False,
                            vertical_flip = True,
                            fill_mode='nearest')

                    test_datagen = ImageDataGenerator(rescale=1./255) # za testiranje fotografije le normaliziramo

                    train_generator = train_datagen.flow_from_directory(
                        train_datadir,
                        target_size = (imsizee,imsizee),
                        batch_size = batch_size,
                        class_mode='binary'
                        )

                    validation_generator = test_datagen.flow_from_directory(
                            validation_datadir,
                            target_size=(imsizee, imsizee),
                            batch_size=batch_size,
                            class_mode='binary')


                    if K.image_data_format() == 'channels_first':
                        input_shape = (3, imsizee, imsizee)
                    else:
                        input_shape = (imsizee, imsizee, 3)

                    model = Sequential()

                    for _ in range(number_of_conv_layer):
                        model.add(Conv2D(size_of_conv_layer, (3, 3), input_shape=input_shape))
                        model.add(Activation('relu'))
                        model.add(MaxPooling2D(pool_size=(2, 2)))


                    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

                    model.add(Dense(size_of_dense_layer))
                    model.add(Activation('relu'))

                    model.add(Dropout(0.5))

                    model.add(Dense(1))
                    model.add(Activation('sigmoid'))

                    model.compile(loss='binary_crossentropy',
                                  optimizer='rmsprop',
                                  metrics=['accuracy'])

                    tensorboard = TensorBoard(log_dir="logs2/first_try16_batch_size_{}_size_of_dense_layer_{}_number_of_conv_layers_{}_imsizee_{}_size_of_conv_layer_{}".format(batch_size, size_of_dense_layer, number_of_conv_layers, imsizee, size_of_conv_layer),
                                              histogram_freq=1)

                 

                    model.fit_generator(
                            train_generator,
                            steps_per_epoch=1652 // batch_size,
                            epochs=65,
                            validation_data=validation_generator,
                            validation_steps=168 // batch_size,
                            callbacks = [tensorboard])

                    model.save_weights('first_try16' + 'batch_size_' + str(batch_size) + 'size_of_dense_layer_' +str(size_of_dense_layer) + 'number_of_conv_layers_' + str(number_of_conv_layers) + 'imsizee_' + str(imsizee) + '.h5')  # always save your weights after training or during training

  0%|          | 0/3 [00:00<?, ?it/s]


------------------------------------------

batch_size: 16  imsizee: 80  number_of_conv_layer: 1  size_of_conv_layer: 16  size_of_dense_layer: 16
Found 1652 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/65
  2/103 [..............................] - ETA: 4:42 - loss: 0.9758 - accuracy: 0.6875 

In [ ]:

%tensorboard --logdir logs2  # show tensorBoard 